In [10]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
from sklearn.ensemble import RandomForestRegressor
from datetime import date
import time

In [12]:
#used for inserting your username and credentials that you can obtain from spotify developer
client_id= '401f60a70e6a496185c36ede2b49d88a'
client_secret= 'c49a4bfe7e224ff3aa774a7e7b4eacba'

redirect_uri='http://localhost:8910/callback'
username= '236hv7ck8vhp8u4btj9bn0261'

scope = 'user-top-read'


In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                      client_secret=client_secret)
                                                      
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

token = util.prompt_for_user_token(username,scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)